In [ ]:
import yfinance as yf

tickers = ['TSLA', 'SPY', 'BND']
data = yf.download(tickers, start='2019-01-01', end='2025-08-01')['Adj Close']
returns = data.pct_change().dropna()


In [ ]:
# TSLA: Expected return from forecast
tsla_predicted = future_series
tsla_return = (tsla_predicted[-1] - tsla_predicted[0]) / tsla_predicted[0]
tsla_expected_annual_return = tsla_return  # assuming 1-year forecast

# SPY & BND: Historical mean daily return, annualized
spy_bnd_returns = returns[['SPY', 'BND']]
mean_daily_returns = spy_bnd_returns.mean()
spy_expected = mean_daily_returns['SPY'] * 252
bnd_expected = mean_daily_returns['BND'] * 252

expected_returns = pd.Series([tsla_expected_annual_return, spy_expected, bnd_expected], index=['TSLA', 'SPY', 'BND'])
cov_matrix = returns[['TSLA', 'SPY', 'BND']].cov() * 252  # Annualized


In [ ]:
from portfolio_optimizer import generate_portfolios, optimize_portfolios, plot_efficient_frontier

# Generate portfolios
results_df, weights_df = generate_portfolios(expected_returns, cov_matrix)

# Optimize
opt_sharpe, opt_vol = optimize_portfolios(expected_returns, cov_matrix)

# Plot
plot_efficient_frontier(results_df, opt_sharpe, opt_vol, expected_returns)


In [ ]:
def summarize_portfolio(opt_result, expected_returns, cov_matrix):
    weights = opt_result.x
    ret, vol, sharpe = portfolio_stats(weights, expected_returns, cov_matrix)
    summary = pd.DataFrame({
        'Weights': weights
    }, index=expected_returns.index)
    print(summary)
    print(f"\nExpected Annual Return: {ret:.2%}")
    print(f"Volatility: {vol:.2%}")
    print(f"Sharpe Ratio: {sharpe:.2f}")
    return summary

print("🔴 Maximum Sharpe Ratio Portfolio")
summarize_portfolio(opt_sharpe, expected_returns, cov_matrix)

print("\n🔵 Minimum Volatility Portfolio")
summarize_portfolio(opt_vol, expected_returns, cov_matrix)
